In [ ]:
import pandas as pd
import yfinance as yf

In [ ]:
#ASX Tickers
RAW_TICKER_ASX = pd.read_csv('https://www.asx.com.au/asx/research/ASXListedCompanies.csv',skiprows=1)
RAW_TICKER_ASX["Ticker"] = RAW_TICKER_ASX['ASX code']+".AX"
TICKER_ASX = list(RAW_TICKER_ASX['Ticker'].dropna())

In [ ]:
# Nasdaq Tickers
#ftp.nasdaqtrader.com/SymbolDirectory/nasdaqlisted.txt
#ftp.nasdaqtrader.com/SymbolDirectory/otherlisted.txt

RAW_TICKER_NDQ = pd.read_csv('/Users/JZ/Downloads/nasdaqlisted.txt',sep="|")
RAW_TICKER_NDQ["Ticker"] = RAW_TICKER_NDQ['Symbol']
TICKER_NDQ = list(RAW_TICKER_NDQ['Ticker'].dropna())

In [ ]:
#London Stock Exchange Tickers
RAW_TICKER_LSE = pd.read_excel (r'https://docs.londonstockexchange.com/sites/default/files/documents/list_of_sets_securities_14.xls', sheet_name='SETS',skiprows=3)
RAW_TICKER_LSE["Ticker"] = RAW_TICKER_LSE['Mnemonic']+".L"
TICKER_LSE = list(RAW_TICKER_LSE['Ticker'].dropna())

In [ ]:
#NYSE Tickers
#RAW_TICKER_NYSE = pd.read_excel (r'https://www.theice.com/publicdocs/data/NYSE_Equity_Index_Ticker_List.xlsx', sheet_name='SETS',skiprows=3)


In [ ]:
#Append list for data extraction layer
TICKER_ALL = (TICKER_ASX + TICKER_NDQ + TICKER_LSE)
#Remove Duplicates
TICKER_ALL = list(dict.fromkeys(TICKER_ALL))

In [ ]:
#Download part 1: Company Information

import time
start_time = time.time()

company_df = pd.DataFrame()

def Info_Extract(TickName):
    
    global company_df
    
    try:
        tick = yf.Ticker(TickName)
        filter_df = tick.info
        company_df = company_df.append(filter_df, ignore_index=True) 
        print("Completed",TickName)
    except Exception:
        print("Error with",TickName)
        pass

    return()


for company in TICKER_ALL:
    Info_Extract(company)

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#Download part 2: financial data

import time
start_time = time.time()

FINANCIALS_DF = pd.DataFrame()

def Financials_Extract(TickName):
    
    global FINANCIALS_DF

    try:
        tick = yf.Ticker(TickName) 
        
        #income statement
        FIN_OG = tick.financials
        FIN_OG2 = FIN_OG.reset_index()
        FIN_OG2.rename(columns = {'index':'Metric'}, inplace = True)
        FIN_OG3=FIN_OG2.melt(id_vars=["Metric"],var_name="Date",value_name="Value")
        FIN_OG3['TickName']=TickName
        FIN_OG3['Financial Data Type']="Income Statement"
        
        FINANCIALS_DF = FINANCIALS_DF.append(FIN_OG3, ignore_index=True)
         
        #balance sheet
        BS_OG = tick.balance_sheet
        BS_OG2 = BS_OG.reset_index()
        BS_OG2.rename(columns = {'index':'Metric'}, inplace = True)
        BS_OG3=BS_OG2.melt(id_vars=["Metric"],var_name="Date",value_name="Value")
        BS_OG3['TickName']=TickName
        BS_OG3['Financial Data Type']="Balance Sheet"
        
        FINANCIALS_DF = FINANCIALS_DF.append(BS_OG3, ignore_index=True)
        
        #cashflow
        CF_OG = tick.cashflow
        CF_OG2 = CF_OG.reset_index()
        CF_OG2.rename(columns = {'index':'Metric'}, inplace = True)
        CF_OG3=CF_OG2.melt(id_vars=["Metric"],var_name="Date",value_name="Value")
        CF_OG3['TickName']=TickName
        CF_OG3['Financial Data Type']="Cashflow"
        
        FINANCIALS_DF = FINANCIALS_DF.append(CF_OG3, ignore_index=True)
        
            
        print("finished",TickName)
        
    except Exception:
        print("Error with",TickName)
        pass

    return()


for company in TICKER_ALL:
    Financials_Extract(company)

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#Download part 3: Download historical share prices


import time
start_time = time.time()

HIST_PRICE_DF = pd.DataFrame()

def Prices_Extract(TickName):
    
    global HIST_PRICE_DF

    try:
        tick = yf.Ticker(TickName) 
        
        hist_p = tick.history(period="6mo")
        hist_p2 = hist_p.reset_index()
        hist_p2.rename(columns = {'index':'Metric'}, inplace = True)
        
        hist_p2['TickName']=TickName

        HIST_PRICE_DF = HIST_PRICE_DF.append(hist_p2, ignore_index=True)
        
            
        print("finished",TickName)
        
    except Exception:
        print("Error with",TickName)
        pass

    return()


for company in TICKER_ALL:
    Prices_Extract(company)

print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# Export all 3 sections to excel

for company in ASX_TICKER_LIST:
    Info_Extract(company)
INF_DF.to_excel('/Users/JZ/Downloads/TEST_INFO.xlsx')


for company in ASX_TICKER_LIST:
    Financials_Extract(company)
FINANCIALS_DF.to_excel('/Users/JZ/Downloads/TEST_FIN.xlsx')


for company in ASX_TICKER_LIST:
    Prices_Extract(company)
HIST_PRICE_DF.to_excel('/Users/JZ/Downloads/TEST_Prices.xlsx')  



In [ ]:
Need to add section for loading into sqlite database